<a href="https://colab.research.google.com/github/profteachkids/CHE2064_Spring2022/blob/main/DistillationNRTL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che.ipynb
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/FlattenWrap.ipynb
!pip install importnb

In [2]:
from importnb import Notebook
with Notebook(): 
    from FlattenWrap import DotDict, Range, RangeArray, Comp, CompArray, dtox
    from che import Props

import numpy as np
from scipy.optimize import root

In [3]:
p=Props(['Ethanol','Water'])

In [13]:
d=DotDict()

d.P = 1e5
d.N = 10
d.NC = p.N_comps
d.F= 100.
d.Fz = np.array([0.2,0.8])
d.FT =320.
d.D = 0.25*d.F
d.B = d.F - d.D

d.F_stage = d.N // 2

d.reflux_ratio = 10.


In [14]:
def boiling_points(P):
    return root(lambda T: np.diagonal(p.Pvap(T)) - P, p.Tb).x

In [15]:
d.bp = boiling_points(d.P)
bp_scaled = (d.bp - np.mean(d.bp))/1.2 + np.mean(d.bp)
bp_minmax = np.min(d.bp), np.max(d.bp)

d.T = RangeArray(np.linspace(*bp_scaled,d.N), *bp_minmax)
d.TC = Range(bp_scaled[-1], *bp_minmax)

Lguess = np.repeat(d.D*(d.reflux_ratio),d.N)
Lguess[d.F_stage:]+=d.F

d.L = RangeArray(Lguess, 0., 2*d.F*d.reflux_ratio)
d.V = RangeArray(np.repeat(d.D*(d.reflux_ratio+1),d.N), 0., 2*d.F*d.reflux_ratio)

QBGuess = d.D * d.reflux_ratio* np.mean(p.HvapNB)
d.QB = Range(QBGuess, 0., 2*QBGuess )
d.QC = Range(QBGuess, 0., 2*QBGuess)

d.Lx = CompArray(np.tile(d.Fz,(d.N,1)))
d.Vy = CompArray(np.tile(d.Fz,(d.N,1)))



In [16]:
wrap, x, d2, xtod, xtodunk  = dtox(d)

In [17]:
r = DotDict()

r.MB = np.zeros((d.N,d.NC))
zeros = np.zeros((1,p.N_comps))
def eqs(d):
    r.EQUIL = d.Lx * p.NRTL_gamma(d.Lx,d.T)* p.Pvap(d.T) - d.Vy*d.P


    r.Lin = np.r_[(d.L[0]*d.Vy[0])[None,:], d.L[1:,None]*d.Lx[:-1]]

    r.Lout = np.r_[d.L[1:,None]*d.Lx[:-1], (d.B*d.Lx[-1])[None,:]]

    r.Vin = np.r_[d.V[1:,None]*d.Vy[1:], zeros]
    r.Vout = d.V[:,None]*d.Vy

    r.MB = r.Lin - r.Lout + r.Vin - r.Vout

    r.MB[d.F_stage-1] += d.F*d.Fz

    r.TCBP = np.atleast_1d(np.sum(d.Vy[0]* p.NRTL_gamma(d.Vy[0],d.TC) * p.Pvap(d.TC)) - d.P)

    r.T = np.insert(d.T,0,d.TC)
    r.EB = p.Hl(r.Lin, r.T[:-1]) + p.Hv(r.Vin, r.T[1:]) - p.Hl(r.Lout, r.T[1:]) - p.Hv(r.Vout, r.T[1:])
    r.EB[d.F_stage-1] += p.Hl(d.F*d.Fz, d.FT)
    r.EB[-1]+=d.QB

    r.CONDENSER = np.atleast_1d(p.Hv(r.Vout[0],r.T[1]) - p.Hl(r.Vout[0], r.T[0]) - d.QC)

    r.RF = np.atleast_1d((d.V[0]-d.D)/d.D) - d.reflux_ratio
    return np.concatenate([np.ravel(r.EQUIL), np.ravel(r.MB), np.ravel(r.EB), r.TCBP, r.RF, r.CONDENSER]), r


In [18]:
eqs_wrapped = wrap(eqs)

In [19]:
sol=root(eqs_wrapped, x, )

In [20]:
res=xtod(sol.x,d2)

In [21]:
eqs(xtod(sol.x,d2))

(array([-2.76486389e-10, -7.27595761e-11,  3.20142135e-10,  3.27418093e-11,
        -6.83940016e-10,  1.45519152e-11, -5.67524694e-10,  1.63709046e-10,
        -1.89174898e-10, -1.74622983e-10,  5.09317033e-11,  2.18278728e-10,
        -9.27684596e-11, -3.78349796e-10, -1.99634087e-10,  4.65661287e-10,
        -2.73985279e-11, -5.82076609e-11,  6.22349717e-07, -5.45260264e-08,
         8.52651283e-14,  2.84217094e-14, -5.68434189e-14, -2.13162821e-14,
         0.00000000e+00, -2.84217094e-14,  0.00000000e+00,  4.26325641e-14,
         5.68434189e-14, -1.13686838e-13, -4.26325641e-14,  1.70530257e-13,
        -1.42108547e-14, -1.42108547e-13,  4.79616347e-14, -5.68434189e-14,
        -1.69808612e-12,  1.64845915e-12, -3.90167412e-11,  3.92219590e-11,
        -4.47034836e-08,  2.98023224e-08,  2.98023224e-08,  0.00000000e+00,
        -3.72529030e-09, -4.47034836e-08,  7.45058060e-08,  0.00000000e+00,
        -1.04308128e-07, -4.54485416e-07, -3.20142135e-10,  0.00000000e+00,
         1.1